In [28]:
import wrds
import pandas as pd
import datetime

# Connect to WRDS
db = wrds.Connection()

# Function to get S&P 500 and company returns
def get_sp500_and_company_returns():
    # Get the list of S&P 500 companies
    sp500_constituents = db.get_table('comp', 'idxcst_his', columns=['gvkey', 'iid', '"from"', '"thru"'])
    sp500_constituents = sp500_constituents[sp500_constituents['iid'] == '01']
    
    # Get the S&P 500 index returns
    sp500_returns = db.raw_sql("""
        SELECT caldt AS date, vwretd AS ret
        FROM crsp.dsp500
        WHERE caldt BETWEEN '2000-01-01' AND '2010-12-31'
    """)
    
    # Filter for Wednesdays
    sp500_returns['date'] = pd.to_datetime(sp500_returns['date'])
    sp500_returns = sp500_returns[sp500_returns['date'].dt.weekday == 2]
    
    # Initialize an empty DataFrame
    df = pd.DataFrame(columns=['Ticker', 'Date', 'Company Return', 'S&P 500 Return'])
    
    # Loop through each company in the S&P 500
    for _, row in sp500_constituents.iterrows():
        gvkey = row['gvkey']
        
        # Get the lpermno for the gvkey
        permno_data = db.raw_sql(f"""
            SELECT lpermno
            FROM crsp.ccmxpf_linktable
            WHERE gvkey = '{gvkey}' AND linktype IN ('LU', 'LC') AND
                  linkdt <= '2010-12-31' AND (linkenddt IS NULL OR linkenddt >= '2000-01-01')
        """)
        
        if permno_data.empty:
            continue
        
        lpermno = int(permno_data.iloc[0]['lpermno'])
        
        # Get the company returns
        company_returns = db.raw_sql(f"""
            SELECT date, ret
            FROM crsp.dsf
            WHERE permno = {lpermno} AND date BETWEEN '2000-01-01' AND '2010-12-31'
        """)
        
        # Filter for Wednesdays
        company_returns['date'] = pd.to_datetime(company_returns['date'])
        company_returns = company_returns[company_returns['date'].dt.weekday == 2]
        
        # Merge with S&P 500 returns
        merged = pd.merge(company_returns, sp500_returns, on='date', suffixes=('_company', '_sp500'))
        merged['Ticker'] = gvkey
        merged = merged[['Ticker', 'date', 'ret_company', 'ret_sp500']]
        merged.columns = ['Ticker', 'Date', 'Company Return', 'S&P 500 Return']
        
        # Append to the main DataFrame
        df = pd.concat([df, merged])
    
    return df

# Get the DataFrame
df = get_sp500_and_company_returns()
print(df)

Enter your WRDS username [tonyd]:apappas
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
    Ticker       Date  Company Return  S&P 500 Return
0   001013 2000-01-05       -0.030710        0.000852
1   001013 2000-01-12        0.036483       -0.004256
2   001013 2000-01-19       -0.013457        0.001418
3   001013 2000-01-26       -0.067708       -0.005067
4   001013 2000-02-02        0.002715       -0.000552
..     ...        ...             ...             ...
22  184996 2010-12-01       -0.027739        0.021595
23  184996 2010-12-08        0.025665        0.003878
24  184996 2010-12-15        0.037504       -0.004986
25  184996 2010-12-22        0.011469        0.003422
26  184996 2010-12-29        0.049981        0.001216

[228563 rows x 4 columns]


In [33]:
import datetime
import statsmodels.api as sm

def calculate_betas(df):
    df['Year'] = df['Date'].dt.year
    betas = []

    for ticker in df['Ticker'].unique():
        ticker_data = df[df['Ticker'] == ticker]
        for year in ticker_data['Year'].unique():
            yearly_data = ticker_data[ticker_data['Year'] == year]
            if len(yearly_data) > 1:  # Ensure there is enough data to perform regression
                X = sm.add_constant(yearly_data['S&P 500 Return'])
                y = yearly_data['Company Return']
                model = sm.OLS(y, X).fit()
                beta = model.params['S&P 500 Return']
                betas.append({'Ticker': ticker, 'Year': year, 'Beta': beta})
    
    beta_df = pd.DataFrame(betas)
    return beta_df

# Calculate the betas
beta_df = calculate_betas(df)
print(beta_df)

      Ticker  Year      Beta
0     001013  2000  1.463464
1     001013  2001  3.168675
2     001013  2002  2.221561
3     001013  2003  1.776520
4     001013  2004  1.855500
...      ...   ...       ...
3588  180711  2010  1.396892
3589  183377  2009       NaN
3590  183377  2010  1.062764
3591  184725  2010  1.867726
3592  184996  2010  0.715417

[3593 rows x 3 columns]


In [34]:
beta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3593 entries, 0 to 3592
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Ticker  3593 non-null   object 
 1   Year    3593 non-null   int32  
 2   Beta    3557 non-null   float64
dtypes: float64(1), int32(1), object(1)
memory usage: 70.3+ KB


In [40]:
db = wrds.Connection()
ratios = db.raw_sql("""
    SELECT gvkey, datadate, tic, 
           am AS bm, dm AS gpm, cfm, roe, debt_ebitda, inv_turn, ap_turn, ar_turn
    FROM comp.funda
    WHERE datadate BETWEEN '2000-01-01' AND '2010-12-31'
""")
ratios['Year'] = pd.to_datetime(ratios['datadate']).dt.year
merged_df = pd.merge(beta_df, ratios, left_on=['Ticker', 'Year'], right_on=['tic', 'Year'], how='left')
merged_df = merged_df.drop(columns=['gvkey', 'datadate', 'tic'])
merged_df = merged_df.rename(columns={
    'bm': 'Book_to_Market',
    'gpm': 'Gross_Profit_Margin',
    'cfm': 'Cash_Flow_Margin',
    'roe': 'Return_on_Equity',
    'debt_ebitda': 'Debt_to_EBITDA',
    'inv_turn': 'Inventory_Turnover',
    'ap_turn': 'Accounts_Payable_Turnover',
    'ar_turn': 'Receivable_Turnover'
})
merged_df

Enter your WRDS username [tonyd]:apappas
Enter your password:········


OperationalError: (psycopg2.OperationalError) connection to server at "wrds-pgdata.wharton.upenn.edu" (165.123.60.118), port 9737 failed: FATAL:  too many connections for role "apappas"

(Background on this error at: https://sqlalche.me/e/20/e3q8)